In [4]:
import pandas as pd
import sqlalchemy as sa  # Corrigido
from sqlalchemy import create_engine  # Importação correta para criar o engine
import os
import sys
sys.path.append(r'c:\Projetos\Python\JornadaDeDados')

pd.set_option('display.max_columns', None)

In [5]:
df = pd.read_csv("microdados_enem_2023/DADOS/MICRODADOS_ENEM_2023.csv", delimiter=";", encoding="latin-1")

In [6]:
df.columns.values

array(['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE',
       'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO',
       'IN_TREINEIRO', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC',
       'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC',
       'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA',
       'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'SG_UF_PROVA',
       'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_MT', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH',
       'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TP_LINGUA',
       'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC',
       'TX_GABARITO_MT', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1',
       'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
       'NU_NOTA_REDACAO', 'Q001', 'Q002', '

In [7]:
colunasDadosRedacao = ['NU_INSCRICAO','TP_STATUS_REDACAO', 'NU_NOTA_COMP1',
       'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
       'NU_NOTA_REDACAO']

In [8]:
#Aqui estamos separando os dados pelas colunas selecionadas no objeto colunasDadosProvaObj
dadosRedacaoTratamento = df.filter(items=colunasDadosRedacao)

In [9]:
dadosRedacaoTratamento['ID_PROVA_OBJETIVA'] = range(1, len(dadosRedacaoTratamento) +1)
dadosRedacaoTratamento.head()

,NU_INSCRICAO,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,ID_PROVA_OBJETIVA
0,210059085136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,210059527735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,210061103945,1.0,140.0,200.0,100.0,120.0,140.0,700.0,3
3,210060214087,1.0,140.0,200.0,160.0,180.0,200.0,880.0,4
4,210059980948,1.0,120.0,120.0,120.0,120.0,80.0,560.0,5


In [10]:
#Funcoes para tratar os dados
def editar_tp_status_redacao(tp_status_redacao):
    if tp_status_redacao == 1:
        return 'Sem problemas'
    elif tp_status_redacao == 2:
        return 'Anulada'
    elif tp_status_redacao == 3:
        return 'Cópia Texto Motivador'
    elif tp_status_redacao == 4:
        return 'Em Branco'
    elif tp_status_redacao == 5:
        return 'Fuga ao tema'
    elif tp_status_redacao == 6:
        return 'Não atendimento ao tipo textual'
    elif tp_status_redacao == 7:
        return 'Texto insuficiente'
    else:
        return 'Parte desconectada'
    
def editar_presenca(row):
    nu_nota_comp1 = row['NU_NOTA_COMP1']
    nu_nota_comp2 = row['NU_NOTA_COMP2']
    nu_nota_comp3 = row['NU_NOTA_COMP3']
    nu_nota_comp4 = row['NU_NOTA_COMP4']
    nu_nota_comp5 = row['NU_NOTA_COMP5']
    nu_nota_redacao = row['NU_NOTA_REDACAO']

    nu_nota_comp1 = 0 if (pd.isnull(nu_nota_comp1) or nu_nota_comp1 == '') else nu_nota_comp1
    nu_nota_comp2 = 0 if (pd.isnull(nu_nota_comp2) or nu_nota_comp2 == '') else nu_nota_comp2
    nu_nota_comp3 = 0 if (pd.isnull(nu_nota_comp3) or nu_nota_comp3 == '') else nu_nota_comp3
    nu_nota_comp4 = 0 if (pd.isnull(nu_nota_comp4) or nu_nota_comp4 == '') else nu_nota_comp4
    nu_nota_comp5 = 0 if (pd.isnull(nu_nota_comp5) or nu_nota_comp5 == '') else nu_nota_comp5
    nu_nota_redacao = 0 if (pd.isnull(nu_nota_redacao) or nu_nota_redacao == '') else nu_nota_redacao

    # Retorna uma combinação dos resultados ou apenas o valor de interesse
    return nu_nota_comp1, nu_nota_comp2, nu_nota_comp3, nu_nota_comp4, nu_nota_comp5, nu_nota_redacao

In [11]:
def tratardados(dadosRedacao):
    dadosRedacao['TP_STATUS_REDACAO'] = dadosRedacao['TP_STATUS_REDACAO'].apply(editar_tp_status_redacao)
    dadosRedacao[['NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO']] = dadosRedacaoTratamento.apply(lambda row: pd.Series(editar_presenca(row)), axis=1)
    return dadosRedacao

In [12]:
dadosRedacaoLimpos = tratardados(dadosRedacaoTratamento)

In [13]:
dadosRedacaoLimpos.head()

,NU_INSCRICAO,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,ID_PROVA_OBJETIVA
0,210059085136,Parte desconectada,0.0,0.0,0.0,0.0,0.0,0.0,1
1,210059527735,Parte desconectada,0.0,0.0,0.0,0.0,0.0,0.0,2
2,210061103945,Sem problemas,140.0,200.0,100.0,120.0,140.0,700.0,3
3,210060214087,Sem problemas,140.0,200.0,160.0,180.0,200.0,880.0,4
4,210059980948,Sem problemas,120.0,120.0,120.0,120.0,80.0,560.0,5


In [14]:
dadosRedacaoLimpos.dtypes

NU_INSCRICAO           int64
TP_STATUS_REDACAO     object
NU_NOTA_COMP1        float64
NU_NOTA_COMP2        float64
NU_NOTA_COMP3        float64
NU_NOTA_COMP4        float64
NU_NOTA_COMP5        float64
NU_NOTA_REDACAO      float64
ID_PROVA_OBJETIVA      int64
dtype: object

In [15]:
tabela = 'DadosProvaRedacao'

tipo_dado_map = {
    'int64': 'BIGINT',
    'float64': 'FLOAT',
    'object': 'VARCHAR({})',
    'bool': 'BIT',
    'datetime64[ns]': 'DATETIME'
}

def calcular_max_varchar_colunas(dadosRedacaoLimpos, coluna):
    
    #retornar o comprimento máximo da coluna
    if dadosRedacaoLimpos[coluna].dtype == 'object':
        return dadosRedacaoLimpos[coluna].apply(lambda x: len(str(x))).max()
    return None


def gerar_sql_create_table(dadosRedacaoLimpos, tabela):
    colunas = []
    
    for coluna, tipo in dadosRedacaoLimpos.dtypes.items():
        if str(tipo) in tipo_dado_map:
            tipo_sql = tipo_dado_map[str(tipo)]
            if 'VARCHAR' in tipo_sql:
                max_length = calcular_max_varchar_colunas(dadosRedacaoLimpos, coluna)
                if max_length is None:
                    max_length = 255  # Valor padrão caso não seja possível calcular
                tipo_sql = tipo_sql.format(max_length)
            colunas.append(f"[{coluna}] {tipo_sql}")
        else:
            colunas.append(f"[{coluna}] VARCHAR(MAX)")  # Tipo padrão para casos não mapeados
    
    sql_create = f"CREATE TABLE {tabela} (\n" + ",\n".join(colunas) + "\n);"
    return sql_create

# Gerando o script SQL
script_sql_create = gerar_sql_create_table(dadosRedacaoLimpos, tabela)

# Exibe o script SQL, basta apenas copiar o result e colar no SSMS que será criado a tabela
print(script_sql_create)

CREATE TABLE DadosProvaRedacao (
[NU_INSCRICAO] BIGINT,
[TP_STATUS_REDACAO] VARCHAR(31),
[NU_NOTA_COMP1] FLOAT,
[NU_NOTA_COMP2] FLOAT,
[NU_NOTA_COMP3] FLOAT,
[NU_NOTA_COMP4] FLOAT,
[NU_NOTA_COMP5] FLOAT,
[NU_NOTA_REDACAO] FLOAT,
[ID_PROVA_OBJETIVA] BIGINT
);


In [16]:
#Etapa que insere os dados no banco de dados

from sqlalchemy import create_engine
from app.db_config import database_config

# Construa a string de conexão corretamente formatada
connection_string = (
    f"mssql+pyodbc://{database_config['username']}:{database_config['password']}"
    f"@{database_config['host']}/{database_config['database']}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
)

# Crie o engine do SQLAlchemy
engine = create_engine(connection_string)

try:
    # Insere os dados na tabela 'DadosParticipantes'
    dadosRedacaoLimpos.to_sql('DadosProvaRedacao', con=engine, if_exists='append', index=False)
    print("Dados inseridos com sucesso!")
except Exception as e:
    print("Erro ao inserir os dados:", e)

Dados inseridos com sucesso!
